# Подготовка данных

Далее создается иерархия классов, проводится базовый анализ датасета

In [1]:
%load_ext scalene

LOADING
Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version. To profile in line mode, use
`%scrun [options] statement`. To profile in cell mode, use `%%scalene
[options]` followed by your code.


In [2]:
import pandas as pd

df = pd.read_csv("data/train_40k.csv").dropna(how='any')

# Define a list with sentences (1k - 100k sentences)
train_sentences = df['Text'].tolist()
labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()
labels

array([['grocery gourmet food', 'meat poultry', 'jerky'],
       ['toys games', 'games', 'unknown'],
       ['toys games', 'games', 'unknown'],
       ...,
       ['beauty', 'tools accessories', 'mirrors'],
       ['beauty', 'skin care', 'body'],
       ['beauty', 'fragrance', 'women s']], dtype=object)

~~Балансировка классов предлагает молиться~~. Согласно отчёту, в датасете 0.7% дубликатов, все одного класса

### Создание иерархии

In [3]:
def create_hierarchy(df):
    hierarchy = {}

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        current_level = hierarchy  # Start from the root of the hierarchy

        # Build the path based on the values in the current row
        for col in df.columns:
            label = row[col]
            if pd.notna(label):  # Check if the label is not NaN
                # If the label doesn't exist in the current level, add it
                if label not in current_level:
                    current_level[label] = {}
                # Move down to the next level in the hierarchy
                current_level = current_level[label]

    return hierarchy

hierarchy = {'root': create_hierarchy(df[['Cat1', 'Cat2', 'Cat3']])}

 Обучение TF-IDF и иерархического классификатора
Помимо TF-IDF на unlabeled текстах был дообучен DistilRoberta, но точность на эмбеддингах Bert и на TF-IDF не отличается

In [4]:
pretrain_texts = pd.read_csv("data/unlabeled_150k.csv").dropna(how='any')['Text'].tolist()
tfidf_text = pretrain_texts + train_sentences

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(tfidf_text)
embeddings = tfidf_vectorizer.transform(train_sentences)

In [7]:
from model_serving.app.services.lib.classifiers import TopDownClassifier
from sklearn.tree import DecisionTreeClassifier

base_classifier = DecisionTreeClassifier
top_down_clf = TopDownClassifier(base_classifier, hierarchy)
topdown_start_train_time = time.time()
top_down_clf.fit(embeddings, labels)
topdown_end_train_time = time.time()

In [8]:
df = pd.read_csv("data/val_10k.csv").dropna(how='any')

test_sentences = df['Text'].tolist()
test_labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()

In [9]:
test_embeddings = tfidf_vectorizer.transform(test_sentences)

In [10]:
def predict(embeddings):
    top_down_clf.predict(test_embeddings)

In [13]:
%scrun predict(test_embeddings)

SCRUN MAGIC
